# Association Rule Based Recommender System

### İş Problemi

Türkiye’nin en büyük online hizmet platformu olan Armut, hizmet verenler ile hizmet almak isteyenleri buluşturmaktadır.
Bilgisayarın veya akıllı telefonunun üzerinden birkaç dokunuşla temizlik, tadilat, nakliyat gibi hizmetlere kolayca
ulaşılmasını sağlamaktadır.

Hizmet alan kullanıcıları ve bu kullanıcıların almış oldukları servis ve kategorileri içeren veri setini kullanarak Association
Rule Learning ile ürün tavsiye sistemi oluşturulmak istenmektedir.

### Veri Seti Hikayesi

Veri seti müşterilerin aldıkları servislerden ve bu servislerin kategorilerinden oluşmaktadır. Alınan her hizmetin tarih ve saat
bilgisini içermektedir.
- UserId: Müşteri numarası
- ServiceId: Her kategoriye ait anonimleştirilmiş servislerdir
- CategoryId: Anonimleştirilmiş kategorilerdir
- CreateDate: Hizmetin satın alındığı tarih

### Veriyi Hazırlama

In [3]:
!pip install mlxtend
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

     ---------------------------------------- 1.4/1.4 MB 788.5 kB/s eta 0:00:00


In [4]:
df_ = pd.read_csv("D:Recommendation Systems\\armut_arl\\armut_data.csv")
df = df_.copy()

In [5]:
df.head()

,UserId,ServiceId,CategoryId,CreateDate
0,25446,4,5,2017-08-06 16:11:00
1,22948,48,5,2017-08-06 16:12:00
2,10618,0,8,2017-08-06 16:13:00
3,7256,9,4,2017-08-06 16:14:00
4,25446,48,5,2017-08-06 16:16:00


ServiceID ve CategoryID'yi "_" ile birleştirerek hizmetleri temsil edecek yeni bir değişken oluşturunuz

In [8]:
df["Hizmet"] = df["ServiceId"].astype(str) + "_" + df["CategoryId"].astype(str)

In [9]:
df.head()

,UserId,ServiceId,CategoryId,CreateDate,Hizmet
0,25446,4,5,2017-08-06 16:11:00,4_5
1,22948,48,5,2017-08-06 16:12:00,48_5
2,10618,0,8,2017-08-06 16:13:00,0_8
3,7256,9,4,2017-08-06 16:14:00,9_4
4,25446,48,5,2017-08-06 16:16:00,48_5


Veri seti hizmetlerin alındığı tarih ve saatten oluşmaktadır, herhangi bir sepet tanımı (fatura vb. ) bulunmamaktadır. Association Rule
Learning uygulayabilmek için bir sepet (fatura vb.) tanımı oluşturulması gerekmektedir. Burada sepet tanımı her bir müşterinin aylık aldığı
hizmetlerdir. Örneğin; 7256 id'li müşteri 2017'in 8.ayında aldığı 9_4, 46_4 hizmetleri bir sepeti; 2017’in 10.ayında aldığı 9_4, 38_4 hizmetleri
başka bir sepeti ifade etmektedir. Sepetleri unique bir ID ile tanımlanması gerekmektedir. Bunun için öncelikle sadece yıl ve ay içeren yeni bir
date değişkeni oluşturunuz. UserID ve yeni oluşturduğunuz date değişkenini "_" ile birleştirirek ID adında yeni bir değişkene atayınız. 

In [11]:
df["CreateDate"] = pd.to_datetime(df["CreateDate"])

In [15]:
df["New_Date"] = df["CreateDate"].dt.strftime("%Y-%m")

In [17]:
df["SepetId"] = df["UserId"].astype(str) + "_" + df["New_Date"].astype(str)

In [18]:
df.head()

,UserId,ServiceId,CategoryId,CreateDate,Hizmet,New_Date,SepetId
0,25446,4,5,2017-08-06 16:11:00,4_5,2017-08,25446_2017-08
1,22948,48,5,2017-08-06 16:12:00,48_5,2017-08,22948_2017-08
2,10618,0,8,2017-08-06 16:13:00,0_8,2017-08,10618_2017-08
3,7256,9,4,2017-08-06 16:14:00,9_4,2017-08,7256_2017-08
4,25446,48,5,2017-08-06 16:16:00,48_5,2017-08,25446_2017-08


### Birliktelik Kurallarının Üretilmesi ve Öneride Bulunma

In [57]:
service_df = df.groupby(['SepetId', 'Hizmet'])['Hizmet'].count().unstack()

In [62]:
service_df = service_df.fillna(0).applymap(lambda x: 1 if x > 0 else 0)

In [63]:
service_df

Hizmet,0_8,10_9,11_11,12_7,13_11,14_7,15_1,16_8,17_5,18_4,...,46_4,47_7,48_5,49_1,4_5,5_11,6_7,7_3,8_5,9_4
SepetId,,,,,,,,,,,,,,,,,,,,,
0_2017-08,0,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0
0_2017-09,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
0_2018-01,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
0_2018-04,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10000_2017-08,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99_2017-12,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99_2018-01,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99_2018-02,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [66]:
frequent_itemsets = apriori(service_df.astype('bool'),
                            min_support=0.01, #0.01 altındaki değerler gösterilmez
                            use_colnames=True)

In [67]:
frequent_itemsets

,support,itemsets
0,0.019728,(0_8)
1,0.026523,(11_11)
2,0.029374,(12_7)
3,0.056627,(13_11)
4,0.023406,(14_7)
5,0.120963,(15_1)
6,0.014659,(16_8)
7,0.041533,(17_5)
8,0.238121,(18_4)
9,0.045563,(19_6)


In [68]:
rules = association_rules(frequent_itemsets, metric="support", min_threshold=0.01)

In [69]:
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(13_11),(2_0),0.056627,0.130286,0.012819,0.226382,1.737574,0.005442,1.124216,0.449965
1,(2_0),(13_11),0.130286,0.056627,0.012819,0.098394,1.737574,0.005442,1.046325,0.488074
2,(15_1),(2_0),0.120963,0.130286,0.033951,0.280673,2.154278,0.018191,1.209066,0.609539
3,(2_0),(15_1),0.130286,0.120963,0.033951,0.260588,2.154278,0.018191,1.188833,0.616073
4,(15_1),(33_4),0.120963,0.027310,0.011233,0.092861,3.400299,0.007929,1.072262,0.803047


- antecedents : önceki ürün
- consequents : ikinci ürün
- antecedent support : ilk ürünün tek başına gözlenme olasılığı 
- consequent support	 : ikinci ürünün tek başına gözlenme olasılığı
- support : iki ürünün birlikte gözlenme olasılığı 
- consequent support	 : önceki ürün
- confidence : biri alındığında diğerinin alınma olasılığı
- lift : biri satın alındığında diğerinin alınmasının kat sayısı
- levelenge : kaldıraç, supportu yüksek değerlere öncelik verir
- conviction : y olmadan x ürününün beklenen frekansıdır

### arl_recommender fonksiyonu ile en son 2_0 hizmetini alan bir kullanıcıya hizmet önerisinde bulunma

In [70]:
def arl_recommender(rules_df, product_id, rec_count=1):
    #rec_count : yazdırdığımız listenin boyutu, yani kaç öneri istiyoruz
    sorted_rules = rules_df.sort_values("lift", ascending=False)
    recommendation_list = []
    for i, product in enumerate(sorted_rules["antecedents"]):
        for j in list(product):
            if j == product_id:
                recommendation_list.append(list(sorted_rules.iloc[i]["consequents"])[0])

    return recommendation_list[0:rec_count]

In [71]:
arl_recommender(rules,"2_0", 4)

['22_0', '25_0', '15_1', '13_11']